## Exercice 1 : Initialisation de l’environnement et des structures de données ##

In [1]:
import gymnasium as gym
import numpy as np

# Initialisation de l’environnement Taxi-v3
env = gym.make("Taxi-v3")

# Nombre d’états et d’actions
state_size = env.observation_space.n
action_size = env.action_space.n

# Création de la table de politique (probabilité égale pour chaque action)
policy_table = np.ones((state_size, action_size)) / action_size

# Création de la table de valeurs initialisée à zéro
value_table = np.zeros(state_size)

# Affichage des premières lignes des tables
print("Premières lignes de policy_table :\n", policy_table[:5])
print("Premières valeurs de value_table :\n", value_table[:5])


Premières lignes de policy_table :
 [[0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]
 [0.16666667 0.16666667 0.16666667 0.16666667 0.16666667 0.16666667]]
Premières valeurs de value_table :
 [0. 0. 0. 0. 0.]


## Exercice 2 : Exploration et collecte d’épisodes ##

In [2]:
num_episodes = 20

for episode in range(num_episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0
    
    while not done:
        action = env.action_space.sample()  # Choisir une action aléatoire
        next_state, reward, done, _, _ = env.step(action)  # Exécuter l’action
        total_reward += reward
        print(f"Épisode {episode+1}, Action: {action}, Récompense: {reward}")

    print(f"Total des récompenses pour l'épisode {episode+1}: {total_reward}\n")


Épisode 1, Action: 1, Récompense: -1
Épisode 1, Action: 2, Récompense: -1
Épisode 1, Action: 3, Récompense: -1
Épisode 1, Action: 4, Récompense: -10
Épisode 1, Action: 2, Récompense: -1
Épisode 1, Action: 2, Récompense: -1
Épisode 1, Action: 1, Récompense: -1
Épisode 1, Action: 4, Récompense: -10
Épisode 1, Action: 5, Récompense: -10
Épisode 1, Action: 2, Récompense: -1
Épisode 1, Action: 3, Récompense: -1
Épisode 1, Action: 1, Récompense: -1
Épisode 1, Action: 2, Récompense: -1
Épisode 1, Action: 5, Récompense: -10
Épisode 1, Action: 4, Récompense: -10
Épisode 1, Action: 0, Récompense: -1
Épisode 1, Action: 2, Récompense: -1
Épisode 1, Action: 0, Récompense: -1
Épisode 1, Action: 2, Récompense: -1
Épisode 1, Action: 0, Récompense: -1
Épisode 1, Action: 0, Récompense: -1
Épisode 1, Action: 5, Récompense: -10
Épisode 1, Action: 1, Récompense: -1
Épisode 1, Action: 0, Récompense: -1
Épisode 1, Action: 2, Récompense: -1
Épisode 1, Action: 1, Récompense: -1
Épisode 1, Action: 1, Récompense

## Exercice 3 : Mise à jour de la politique avec PPO ##

In [3]:
gamma = 0.99  # Facteur d’actualisation
lr_policy = 0.1  # Taux d’apprentissage pour la politique
clip_epsilon = 0.2  # Seuil de clipping

# Exemple de stockage des épisodes
episode_states = []
episode_actions = []
episode_rewards = []

In [4]:
# Simulation d’un épisode
state, _ = env.reset()
done = False

while not done:
    action = np.random.choice(action_size, p=policy_table[state])  # Sélection de l’action
    next_state, reward, done, _, _ = env.step(action)
    
    episode_states.append(state)
    episode_actions.append(action)
    episode_rewards.append(reward)
    
    state = next_state

# Calcul des récompenses cumulées (discounted rewards)
discounted_rewards = np.zeros_like(episode_rewards, dtype=np.float32)
G = 0
for t in reversed(range(len(episode_rewards))):
    G = episode_rewards[t] + gamma * G
    discounted_rewards[t] = G

# Mise à jour de la table de valeurs et politique
for i, state in enumerate(episode_states):
    advantage = discounted_rewards[i] - value_table[state]
    
    # Mise à jour de la valeur de l’état
    value_table[state] += lr_policy * advantage

    # Mise à jour de la politique avec PPO (avec clipping)
    old_prob = policy_table[state, episode_actions[i]]
    new_prob = old_prob + lr_policy * advantage
    policy_table[state, episode_actions[i]] = np.clip(new_prob, 
                                                      old_prob - clip_epsilon, 
                                                      old_prob + clip_epsilon)

print("Mise à jour de la politique et des valeurs effectuée !")
print("Premières lignes de policy_table après mise à jour :\n", policy_table[:5])


Mise à jour de la politique et des valeurs effectuée !
Premières lignes de policy_table après mise à jour :
 [[ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]
 [ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]
 [ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]
 [-0.23333333 -0.23333333 -0.43333333 -0.03333333 -0.23333333  0.16666667]
 [ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]]


In [5]:
print("Premières lignes de policy_table après mise à jour :\n", policy_table[:5])

Premières lignes de policy_table après mise à jour :
 [[ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]
 [ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]
 [ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]
 [-0.23333333 -0.23333333 -0.43333333 -0.03333333 -0.23333333  0.16666667]
 [ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]]


## Exercice 4: Évaluation de l’agent après entraînement ##

In [8]:
num_eval_episodes = 20
total_rewards = []
max_steps = 10000
steps = 0


for ep in range(num_eval_episodes):
    state, _ = env.reset()
    total_reward = 0
    done = False
    
    while not done and steps < max_steps:
        action = np.argmax(policy_table[state])  # Sélection de l'action optimale
        next_state, reward, done, _, _ = env.step(action)
        total_reward += reward
        state = next_state
        steps += 1
        print(f"État: {state}, Action: {action}, Récompense: {reward}, Done: {done}")
    
    total_rewards.append(total_reward)
    print(f"Épisode {ep+1}: Récompense totale = {total_reward}")

# Affichage des performances moyennes
print(f"Récompense moyenne après entraînement: {np.mean(total_rewards)}")


État: 262, Action: 0, Récompense: -1, Done: False
État: 362, Action: 0, Récompense: -1, Done: False
État: 462, Action: 0, Récompense: -1, Done: False
État: 462, Action: 0, Récompense: -1, Done: False
État: 462, Action: 0, Récompense: -1, Done: False
État: 462, Action: 0, Récompense: -1, Done: False
État: 462, Action: 0, Récompense: -1, Done: False
État: 462, Action: 0, Récompense: -1, Done: False
État: 462, Action: 0, Récompense: -1, Done: False
État: 462, Action: 0, Récompense: -1, Done: False
État: 462, Action: 0, Récompense: -1, Done: False
État: 462, Action: 0, Récompense: -1, Done: False
État: 462, Action: 0, Récompense: -1, Done: False
État: 462, Action: 0, Récompense: -1, Done: False
État: 462, Action: 0, Récompense: -1, Done: False
État: 462, Action: 0, Récompense: -1, Done: False
État: 462, Action: 0, Récompense: -1, Done: False
État: 462, Action: 0, Récompense: -1, Done: False
État: 462, Action: 0, Récompense: -1, Done: False
État: 462, Action: 0, Récompense: -1, Done: False


In [9]:
print("Table de politique pour l'état 411:", policy_table[411])

Table de politique pour l'état 411: [ 0.36666667 -1.23333333 -0.63333333 -0.63333333 -0.43333333  0.36666667]
